### 2.1. OpenAI API 과거 데이터 추가 라벨링 작업

In [59]:
%%writefile /content/news-classifier/notebooks/02_gpt_labeling.ipynb
# ============================================================================
# 1. 필수 패키지 설치 및 라이브러리 임포트
# ============================================================================
print("="*80)
print("🔧 1. 필수 패키지 설치 및 라이브러리 임포트")
print("="*80)
!pip install -q --upgrade datasets polars openai pandas tqdm
print("✅ 패키지 설치 완료\n")

import os
import sys
import time
import numpy as np
import pandas as pd
from pathlib import Path
import shutil
from google.colab import drive
from tqdm.auto import tqdm
from openai import OpenAI, RateLimitError
from datasets import load_dataset
from concurrent.futures import ThreadPoolExecutor

# TQDM이 Pandas와 잘 작동하도록 설정
tqdm.pandas()

# ============================================================================
# 2. Google Drive 마운트 및 경로 설정 (★ 1000개 기준으로 수정)
# ============================================================================
print("\n" + "="*80)
print("📁 2. Google Drive 마운트 및 경로 설정")
print("="*80)

drive.mount('/content/drive', force_remount=False)

# --- 경로 설정 ---
SAVE_BASE_PATH = Path("/content/drive/MyDrive/best_news_classifier")
SAVE_BASE_PATH.mkdir(parents=True, exist_ok=True)
print(f"✅ 기본 저장 경로: {SAVE_BASE_PATH}")

ORIGINAL_DATA_PATH = "/content/drive/MyDrive/joongang_crawl/combined_articles.parquet"
if not Path(ORIGINAL_DATA_PATH).exists():
    ORIGINAL_DATA_PATH = "/content/drive/MyDrive/combined_articles.parquet"
if not Path(ORIGINAL_DATA_PATH).exists():
     raise FileNotFoundError(f"❌ '전체' 원본 데이터 파일을 찾을 수 없습니다: {ORIGINAL_DATA_PATH}")

# (입력) 기존 500개 샘플 파일
EXISTING_500_PATH = SAVE_BASE_PATH / "chatgpt_500_labels_1990_2019.parquet"
if not EXISTING_500_PATH.exists():
     raise FileNotFoundError(f"❌ 500개 샘플 파일({EXISTING_500_PATH.name})을 찾을 수 없습니다.")

# (★수정★) (출력) 500개를 추가하여 새로 저장할 1000개 샘플 파일
FINAL_1000_PATH = SAVE_BASE_PATH / "chatgpt_1000_labels_1990_2019.parquet"
print(f"✅ 기존 500개 파일: {EXISTING_500_PATH.name}")
print(f"✅ 최종 1000개 파일: {FINAL_1000_PATH.name}")


# ============================================================================
# 3. '전체' 원본 데이터 로드 및 1990-2019년 필터링
# ============================================================================
print("\n" + "="*80)
print("🚀 3. '전체' 원본 데이터 로드 및 필터링")
print("="*80)

LOCAL_DATA_PATH = Path("/dev/shm/combined_articles.parquet")
if not LOCAL_DATA_PATH.exists():
    print(f"🚀 전체 데이터 복사 시작: Drive -> RAM Disk (/dev/shm/)...")
    shutil.copyfile(ORIGINAL_DATA_PATH, LOCAL_DATA_PATH)
    print("✅ 데이터 복사 완료")
else:
    print("✅ (스킵) RAM 디스크에 '전체' 데이터가 이미 존재합니다.")

print(f"... RAM 디스크에서 {LOCAL_DATA_PATH.name} 로드 중 ...")
all_data_dataset = load_dataset("parquet", data_files={"inference": str(LOCAL_DATA_PATH)}, split="inference")
print(f"✅ '전체' 데이터 로드 완료: {len(all_data_dataset):,} 건")

print(f"... {len(all_data_dataset):,} 건의 전체 데이터에서 1990~2019년 데이터 필터링 중 ...")
def filter_past_data(example):
    try:
        yyyymmdd = float(example['yyyymmdd'])
        return 19900101 <= yyyymmdd <= 20191231
    except (TypeError, ValueError, AttributeError):
        return False

past_data_dataset = all_data_dataset.filter(filter_past_data, num_proc=4)
print(f"✅ 1990~2019년 데이터 {len(past_data_dataset):,} 건 필터링 완료.")

# (★수정) 총 1000개(500+500)가 필요하므로, 1000개 미만인지 체크
if len(past_data_dataset) < 1000:
     raise ValueError(f"필터링된 1990-2019년 데이터가 1000개 미만입니다 ({len(past_data_dataset)}건).")


# ============================================================================
# 4. [★수정★] 500개 신규 샘플 추출 (중복 제외)
# ============================================================================
print("\n" + "="*80)
print("🎲 4. 500개 신규 샘플 추출 (중복 제외)")
print("="*80)

df_existing_500 = pd.read_parquet(EXISTING_500_PATH)
print(f"✅ 기존 500개 샘플 로드 완료.")

shuffled_dataset = past_data_dataset.shuffle(seed=42)

# (★수정★) 500개 이후의 500개를 선택 (range(500, 1000))
print("... 1990-2019 데이터에서 500~1000번째 샘플 500개 추출 중 ...")
new_samples_dataset = shuffled_dataset.select(range(500, 1000))
new_samples_df = new_samples_dataset.to_pandas()
print(f"✅ 신규 500개 샘플 추출 완료. (총 1,000개 샘플 준비)")


# ============================================================================
# 5. ChatGPT API 병렬 호출 (★수정★ 500건)
# ============================================================================
print("\n" + "="*80)
print("🤖 5. ChatGPT API 병렬 호출 (신규 500건)")
print("="*80)

# --- OpenAI API 설정 ---
from google.colab import userdata
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    llm_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    llm_model = "gpt-4o-mini"
except Exception as e:
    print("❌ Colab '비밀' 탭에서 'OPENAI_API_KEY'를 설정하세요! (오류: {e})")
    sys.exit(1)

CATEGORIES = "[경제, 정치, 사회, 국제, 문화/스포츠, 기타]"
SYSTEM_PROMPT = f"""당신은 한국어 뉴스 기사 분류 전문가입니다.
기사의 헤드라인과 본문을 읽고, 6개의 카테고리 중 가장 적합한 것 하나로 분류합니다.
6개 카테고리는 {CATEGORIES} 입니다.
반드시 6개 카테고리 중 하나로만 대답해야 하며, 그 외의 설명이나 문장은 절대 포함하지 마십시오."""

def get_chatgpt_label_with_retry(headline, content, max_retries=5):
    user_prompt = f"""[헤드라인]: {headline}
[본문 요약]: {str(content)[:1000]}
[분류]: """
    messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]
    result = {'label': '분류오류', 'prompt_tokens': 0, 'completion_tokens': 0}
    retries = 0
    wait_time = 5
    while retries < max_retries:
        try:
            response = llm_client.chat.completions.create(
                model=llm_model, messages=messages, temperature=0.0, max_tokens=20
            )
            answer = response.choices[0].message.content.strip()
            if answer in ['경제', '정치', '사회', '국제', '문화/스포츠', '기타']:
                result['label'] = answer
            else:
                result['label'] = "분류실패"
            if response.usage:
                result['prompt_tokens'] = response.usage.prompt_tokens
                result['completion_tokens'] = response.usage.completion_tokens
            return result
        except RateLimitError as e:
            retries += 1
            if retries == max_retries:
                print(f"🚨 API 최대 재시도(5회) 실패. [오류] {e}")
                return result
            print(f"🕒 429 RateLimitError. {wait_time}초 후 재시도... ({retries}/{max_retries})")
            time.sleep(wait_time)
            wait_time *= 2
        except Exception as e:
            print(f"🚨 API 기타 오류. [오류] {e}")
            return result

# --- 병렬 처리 (ThreadPoolExecutor) ---
MAX_WORKERS = 10
results = []

print(f"... [병렬 처리 시작] 신규 500개 샘플 API 호출 (Workers={MAX_WORKERS}) ...")
print("... 429 에러 발생 시 자동으로 재시도합니다 ...")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    tasks_iter = ( (row['headline'], row['content']) for index, row in new_samples_df.iterrows() )
    results = list(tqdm(
        executor.map(lambda p: get_chatgpt_label_with_retry(p[0], p[1]), tasks_iter),
        total=len(new_samples_df), # (★수정) 500개
        desc="API 병렬 호출"
    ))

print("✅ 신규 500개 API 호출 완료!")


# ============================================================================
# 6. [★수정★] 500개(기존) + 500개(신규) 병합 및 저장
# ============================================================================
print("\n" + "="*80)
print("💾 6. 500개(기존) + 500개(신규) 병합 및 저장")
print("="*80)

# 1. 500개 신규 결과(List[Dict])를 DataFrame으로 변환
df_new_results = pd.json_normalize(results)

# 2. 원본 500개 샘플(new_samples_df)과 신규 결과(df_new_results) 결합
df_new_labeled_500 = pd.concat([new_samples_df.reset_index(drop=True), df_new_results], axis=1)

# 3. 'label' -> 'chatgpt_label'로 이름 변경
df_new_labeled_500.rename(columns={'label': 'chatgpt_label'}, inplace=True)

# 4. 기존 500개(df_existing_500)와 신규 500개(df_new_labeled_500) 최종 병합
df_total_1000 = pd.concat([df_existing_500, df_new_labeled_500], ignore_index=True)

print(f"✅ 기존 500개 + 신규 500개 = 총 {len(df_total_1000)}개 데이터 병합 완료.")

# 5. (★수정★) 최종 1,000개 파일을 Drive에 저장
df_total_1000.to_parquet(FINAL_1000_PATH, index=False)
print(f"💾 최종 1,000개 샘플이 {FINAL_1000_PATH.name} 파일로 저장되었습니다.")


# ============================================================================
# 7. 신규 500건에 대한 토큰 사용량 및 비용 계산
# ============================================================================
print("\n" + "="*80)
print("💰 7. 신규 500건에 대한 토큰 사용량 및 비용 계산")
print("="*80)

PRICE_INPUT_PER_1M_TOKENS = 0.15  # $0.15
PRICE_OUTPUT_PER_1M_TOKENS = 0.60 # $0.60

# (★수정★) 'df_new_labeled_500' (신규 500개) 기준으로만 계산
total_prompt_tokens = df_new_labeled_500['prompt_tokens'].sum()
total_completion_tokens = df_new_labeled_500['completion_tokens'].sum()
total_tokens = total_prompt_tokens + total_completion_tokens

cost_input = (total_prompt_tokens / 1_000_000) * PRICE_INPUT_PER_1M_TOKENS
cost_output = (total_completion_tokens / 1_000_000) * PRICE_OUTPUT_PER_1M_TOKENS
total_cost = cost_input + cost_output

print(f"--- [신규 500건 작업 결과] ---")
print(f"총 사용 토큰: {total_tokens:,} 개")
print(f" - 입력 (Prompt) 토큰: {total_prompt_tokens:,} 개")
print(f" - 출력 (Completion) 토큰: {total_completion_tokens:,} 개")
print("\n--- 예상 비용 (USD) ---")
print(f"입력 비용: ${cost_input:.6f}")
print(f"출력 비용: ${cost_output:.6f}")
print(f"총 비용:   ${total_cost:.6f} (약 {total_cost * 1350:.0f} 원)") # (환율 1350원 가정)
print("="*80)

print("\n🎉 모든 500개 추가 라벨링 작업 완료! 🎉")

Writing /content/news-classifier/notebooks/02_gpt_labeling.ipynb


In [6]:
df_total_1000

,headline,url,sector1,sector2,content,yyyymmdd,chatgpt_label,prompt_tokens,completion_tokens
0,"損保社, 현금배당 줄이어",https://www.joongang.co.kr/article/335433,경제,금융증권,오는 6월 중순 일제히 주주총회를 여는 손해보험사들이 높은 현금배당을 하기로 했다....,20040512,경제,297,1
1,알제리 원자로 건설/중국서 기술등 지원,https://www.joongang.co.kr/article/2555679,정치,정치일반,【워싱턴 로이터=연합】 미국의 정보기관들은 중국이 핵무기 또는 핵연료 생산으로 이어...,19910421,국제,206,2
2,‘인지도 높은’ 조윤선·박선영·박선숙에 ‘장관 출신’ 김장수·송민순도,https://www.joongang.co.kr/article/3280313,중앙SUNDAY,뉴스,상임위 활동은 ‘여의도 스타’로 자리매김할 수 있는 좋은 기회다. 국회 선수(選手)...,20080831,정치,811,2
3,"[저자와 함께] 나의 조상, 있는 그대로 보고 싶었다",https://www.joongang.co.kr/article/388521,문화,책,“집안 얘기를 책으로 내놓고 보니 좀 껄끄럽네요. 다시 쓰라면 못하겠습니다.”\n자...,20040911,문화/스포츠,781,4
4,고농축 비타민 듬뿍···티나코스 마스크팩 2종 나왔다,https://www.joongang.co.kr/article/21764912,사회,사회일반,여성의 대표적인 피부고민인 수분과 톤업을 위해 티나코스가 마스크팩 2종을 런칭 했다...,20170717,기타,640,2
...,...,...,...,...,...,...,...,...,...
995,[브리핑] 부동산 중개업소 위법 60건 적발,https://www.joongang.co.kr/article/14259775,사회,전국,전남도가 봄 이사철을 앞두고 부동산 중개업소를 점검해 총 60건을 시정 조치했다. ...,20140326,사회,295,1
996,<모임>제헌동지회 오찬모임,https://www.joongang.co.kr/article/3486584,정치,정치일반,제49주년 제헌절을 맞아 제헌동지회 오찬모임이 金守漢 국회의장 초청으로 17일 낮1...,19970718,정치,336,2
997,<업계단신>초박형 도체 量産체제 돌입-금성전선,https://www.joongang.co.kr/article/2914427,정치,정치일반,"金星電線 전선부문(대표 權문久)은 카드케이블의 원자재가 되는초박형 도체를 국산화,최...",19940804,경제,270,1
998,'보수냐 당근이냐' 사외이사 대접 논란,https://www.joongang.co.kr/article/3960990,정치,정치일반,서울 모대학 교수인 A씨는 지난해 겨울 딸의 결혼식 때 한 축의금 봉투를 보고 깜짝...,20000826,사회,788,1
